# Lazy sequences working hard

[@thomasguest](https://twitter.com/thomasguest) • [wordaligned.org](http://wordaligned.org) • [@clinithinkwales](https://twitter.com/clinithinkwales)

A lazy sequence defers supplying values to clients until they're needed. This makes it possible to work with large and even infinite sequences using limited memory.

Python provides language and library level support for lazy sequences.

In [1]:
G = 10 ** 100
R = range(G)
R

range(0, 10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000)

In [2]:
def take(xs, n):
    xs = iter(xs)
    result = [next(xs) for _ in range(n)]
    return result

rs = iter(R)
print(take(rs, 10), '\n', next(rs))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9] 
 10


### Procrastination
The call to `range()` creates a range object.
This isn't a collection in memory.
Instead, the object stores start, stop, step values, so it can be iterated over etc.

Activity is deferred until necessary.

This is lazy working, not lazy thinking.

### Historical note 
In Python 2 `range()` created a list and `xrange()` behaved lazily.
Python 3 simplified things: there's no need to have both.
Lazy is sufficient.

In [3]:
list(range(10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

See also: `map()`, `filter()`, `dict.keys()` ...

## Lazy-*ish*
Range objects are not *very* lazy or sequential. They give the illusion of being all there all of the time.
- You can get their length directly
- You can access the value at an index
- You can slice them
- You can check membership

In [4]:
len(R)

OverflowError: Python int too large to convert to C ssize_t

In [5]:
R[666]  

666

In [6]:
R[100:200:25]

range(100, 200, 25)

In [7]:
289 in R[::17]

True

## Generators
Generators offer language level support for lazy lists. 

Generators `yield` values one by one. 

You can step using `next()` or iterate using `for`.

You cannot go backwards, query the length, etc.

In [8]:
# Generator function
import random

def flip_a_coin():
    'Generate a random sequence of Heads/Tails'
    while True:
        yield random.choice(['Heads', 'Tails'])

flip_a_coin()

<generator object flip_a_coin at 0x0000008DA4112888>

In [9]:
ht = flip_a_coin()
print(next(ht), next(ht))
' '.join(next(ht) for _ in range(10))

Tails Heads


'Tails Heads Tails Tails Heads Heads Tails Heads Tails Tails'

In [10]:
M = 1000 * 1000
heads = sum(1 for h, _ in zip(ht, range(M)) if h == 'Heads')
heads, heads/M

(500693, 0.500693)

In [11]:
import collections

last10 = collections.deque(take(ht, 10), maxlen=10)

tosses = enumerate(ht)
print(take(tosses, 5))
while len(set(last10)) == 2:
    last10.append(next(tosses)[1])

next(tosses)

[(0, 'Tails'), (1, 'Heads'), (2, 'Heads'), (3, 'Tails'), (4, 'Heads')]


(2689, 'Heads')

In [12]:
# Generator expressions look like list comprehensions
# but they create generators, not lists
(x * x for x in R)

<generator object <genexpr> at 0x0000008DA43A4938>

In [13]:
sqs = (x * x for x in R)
print(take(sqs,10))
print(next(sqs))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
100


## Standard Generators
Many functions in the Python standard library are generators.

In [14]:
import os

PY = 'C:/Python35'
tree = os.walk(PY)
next(tree)

('C:/Python35',
 ['DLLs', 'Doc', 'include', 'Lib', 'libs', 'Scripts', 'Tools'],
 ['LICENSE.txt',
  'NEWS.txt',
  'python.exe',
  'python.pdb',
  'python3.dll',
  'python35.dll',
  'python35.pdb',
  'python35_d.dll',
  'python35_d.pdb',
  'python3_d.dll',
  'pythonw.exe',
  'pythonw.pdb',
  'pythonw_d.exe',
  'pythonw_d.pdb',
  'python_d.exe',
  'python_d.pdb',
  'README.txt',
  'vcruntime140.dll'])

In [15]:
subtrees = (r for r, d, f in os.walk(PY))
take(subtrees, 5)

['C:/Python35',
 'C:/Python35\\DLLs',
 'C:/Python35\\Doc',
 'C:/Python35\\include',
 'C:/Python35\\Lib']

In [16]:
def ilen(xs):
    "Return the length of an iterable"
    return sum(1 for _ in xs)

all_files = (os.path.join(r, f)
             for r, _, ff in os.walk(PY)
             for f in ff)

py_files = (py for py in all_files if py.endswith('.py'))

max(py_files, key=lambda py: ilen(open(py, errors='ignore')))

'C:/Python35\\Lib\\_pydecimal.py'

## Shell Pipeline
    $ find /c/Python35 -name "*.py" | xargs -L1 wc -l | sort -rn | head -1
    6399 /c/Python35/Lib/_pydecimal.py


## Chunked

In [17]:
rs = iter(R)
r3 = [rs, rs, rs]
[next(r) for r in r3]

[0, 1, 2]

In [18]:
[next(r) for r in r3]

[3, 4, 5]

In [19]:
z = zip(*r3)
take(z, 3)

[(6, 7, 8), (9, 10, 11), (12, 13, 14)]

In [20]:
def chunks(xs, n):  
    return zip(*[iter(xs)]*n)

take(chunks(R[::1000], 2), 5)

[(0, 1000), (2000, 3000), (4000, 5000), (6000, 7000), (8000, 9000)]

## Flattened

In [21]:
import itertools
flatten = itertools.chain
flatten('abc', 'def')

In [22]:
list(_)

['a', 'b', 'c', 'd', 'e', 'f']

In [23]:
xs = chunks(R[::1000], 2)
flatten = itertools.chain.from_iterable
ys = flatten(xs)
take(ys, 5)

[0, 1000, 2000, 3000, 4000]

## Itertools
- my favourite Python module
- composable functions for working with lazy lists
- examples
  - count()
  - islice()
  - cycle()
  - repeat()

In [19]:
import itertools as its
def ilen(xs): return sum(1 for _ in xs)
def run_length_encode(xs):
    ''' Run length encodes the stream of values, xs
    
    The result is a stream of (value, repeat-count) pairs
    '''
    return ((x, ilen(g)) for x, g in its.groupby(xs))
    
rle = run_length_encode(flip_a_coin())
list(its.islice(rle, 5))

[('Tails', 1), ('Heads', 5), ('Tails', 6), ('Heads', 2), ('Tails', 1)]

In [25]:
def run_length_decode(vc):
    '''Reverses run_length_encode
    
    Expands a stream of (value, repeat-count) pairs into 
    a stream of values.
    '''
    return its.chain.from_iterable(its.starmap(its.repeat, vc))

scream = ('A', 6), ('R', 4), ('G', 5), ('H', 3), ('!', 13)
''.join(run_length_decode(scream))

'AAAAAARRRRGGGGGHHH!!!!!!!!!!!!!'

In [23]:
import random

def story_points():
    while True:
        yield int(random.gauss(100, 25))

def smooth(xs, n=10):
    b, e = its.tee(xs)
    s = sum(its.islice(e, n))
    for bn, en in zip(b, e):
        s += en - bn
        yield s / n

take(smooth(story_points(), 10000), 10)

[99.3534,
 99.3528,
 99.3477,
 99.3474,
 99.3492,
 99.3566,
 99.3616,
 99.364,
 99.3631,
 99.3695]

## Emulating Shell Syntax

You have to read the expression:

    take(smooth(story_points(), 10), 10)

inside out.

Wouldn't it be nice to borrow shell syntax?

    story_points() | smooth(10) | take(10)

In [21]:
class pipe:
    def __init__(self, fn):
        self._fn = fn

    def __ror__(self, other):
        return self._fn(other)

    def __call__(self, *v, **k):
        return pipe(lambda x: self._fn(x, *v, **k))

# Credit: https://github.com/JulienPalard/Pipe

In [26]:
psmooth = pipe(smooth)
ptake = pipe(take)
story_points() | psmooth(2) | ptake(10) 

[87.5, 100.0, 105.5, 100.0, 95.5, 100.5, 120.5, 124.5, 111.0, 84.0]

In [ ]:
@pipe
def bounds(xs):
    m, M = 1e9, -1e9
    for x in xs:
        m, M = min(m, x), max(M, x)
        yield m, M

from itertools import *
xs = count()
ys, xs = tee(xs)
take(xs, 10**9)
#next(ys)

# Thank you

[@thomasguest](https://twitter.com/thomasguest) • [wordaligned.org](http://wordaligned.org) • [@clinithinkwales](https://twitter.com/clinithinkwales)